In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
dataset = pd.read_csv('credito4.csv', sep=';')
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [4]:
dataset.shape

(1000, 8)

In [6]:
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]

In [ ]:
# Checkando quantos valores temos nulos
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [10]:
y.isnull().sum()

np.int64(0)

In [11]:
# Usando o fillna para preencher os dados faltantes
mediana_saldo_atual = X['SALDO_ATUAL'].median()
mediana_saldo_atual

np.float64(2323.0)

In [14]:
X['SALDO_ATUAL'].fillna(mediana_saldo_atual, inplace=True)

C:\Users\Igor\AppData\Local\Temp\ipykernel_6040\3955557194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana_saldo_atual, inplace=True)


In [16]:
# Pelo warning assumo que para o futuro devemos usar X.fillna({'SALDO_ATUAL': mediana_saldo_atual}, inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64